-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

## Promoting to Silver

Here we'll pull together the concepts of streaming from Delta Tables, deduplication, and quality enforcement to finalize our approach to our silver table.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_heartrate_silver.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Apply table constraints to Delta Lake tables
- Use flagging to identify records failing to meet certain conditions
- Apply de-duplication within an incremental microbatch
- Use `MERGE` to avoid inserting duplicate records to a Delta Lake table

In [0]:
%run ../Includes/silver-setup

Begin by resetting your table and checkpoint to make sure there are no conflicts from previous writes.

In [0]:
spark.sql("DROP TABLE IF EXISTS heart_rate_silver")

dbutils.fs.rm(Paths.silverRecordingsTable, True)
dbutils.fs.rm(Paths.silverRecordingsCheckpoint, True)

spark.sql(f"""
CREATE TABLE IF NOT EXISTS heart_rate_silver
(device_id LONG, time TIMESTAMP, heartrate DOUBLE, bpm_check STRING)
USING DELTA
LOCATION '{Paths.silverRecordingsTable}'
""")

Add a table constraint before inserting data. Name this constraint `dateWithinRange` and make sure that the time is greater than January 1, 2017.

In [0]:
%sql
MAGIC -- TODO
MAGIC -- ALTER TABLE -- <FILL-IN>

Note that adding and removing constraints is recorded in the transaction log.

In [0]:
%sql
DESCRIBE HISTORY heart_rate_silver

Use the cell below to create a streaming read that includes:
1. A filter for the topic `bpm`
2. Logic to flatten the JSON payload and cast data to the appropriate schema
3. A `bpm_check` column to flag negative records
4. A duplicate check on `device_id` and `time` with a 30 second watermark on `time`

In [0]:
# TODO
streamingDF = (
# <FILL_IN>
)

Below, the upsert class used in the previous notebooks is provided.

In [0]:
class Upsert:
    def __init__(self, query, update_temp="stream_updates"):
        self.query = query
        self.update_temp = update_temp 
        
    def upsertToDelta(self, microBatchDF, batch):
        microBatchDF.createOrReplaceTempView(self.update_temp)
        microBatchDF._jdf.sparkSession().sql(self.query)

Use the cell below to define the upsert query to instantiate our class. Alternately, [consult the documentation](https://docs.databricks.com/delta/delta-update.html#upsert-into-a-table-using-merge&language-python) and try implementing this using the `DeltaTable` Python class.

In [0]:
# TODO
query = """<FILL-IN>"""
 
streamingMerge=Upsert(query)

Now execute a write with trigger once logic to process all existing data from the bronze table.

In [0]:
def process_silver_heartrate():
    (streamingDF.writeStream
       .foreachBatch(streamingMerge.upsertToDelta)
       .outputMode("update")
       .option("checkpointLocation", Paths.silverRecordingsCheckpoint)
       .trigger(once=True)
       .start()
       .awaitTermination())

process_silver_heartrate()

We should see the same number of total records in our silver table as previously, but a small percentage of these will correctly be flagged with "Negative BPM".

In [0]:
%sql
SELECT COUNT(*)
FROM heart_rate_silver

In [0]:
%sql
SELECT COUNT(*)
FROM heart_rate_silver
WHERE bpm_check = "Negative BPM"

Now land a new batch of data and propagate changes through bronze into the silver table.

In [0]:
new_batch()
process_silver_heartrate()

In [0]:
%sql
SELECT COUNT(*)
FROM heart_rate_silver

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>